In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "HmpyE3PpWAgI"
      },
      "source": []
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ZP3NjApxVOVC"
      },
      "source": [
        "# 00 Inital Steps"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "kQUEh4F5NXbx"
      },
      "outputs": [],
      "source": [
        "!pip install keras-unet-collection\n",
        "!pip install -U -q segmentation-models"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 2,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "dSCGeR5sp0AU",
        "outputId": "85504919-15e9-4e26-ade5-5ea3d560c53b"
      },
      "outputs": [
        {
          "name": "stdout",
          "output_type": "stream",
          "text": [
            "Segmentation Models: using `tf.keras` framework.\n"
          ]
        }
      ],
      "source": [
        "import os\n",
        "os.environ[\"SM_FRAMEWORK\"] = \"tf.keras\"\n",
        "import cv2\n",
        "import json\n",
        "import matplotlib.pyplot as plt\n",
        "import numpy as np\n",
        "from glob import glob\n",
        "from google.colab import drive\n",
        "from keras import backend as K\n",
        "from keras.metrics import Precision, Recall, AUC, Accuracy\n",
        "from keras_unet_collection import losses, models, utils\n",
        "import segmentation_models as sm\n",
        "from skimage import measure\n",
        "from sklearn.utils import shuffle\n",
        "from tensorflow import keras\n",
        "from tensorflow.keras.callbacks import EarlyStopping\n",
        "import tensorflow as tf\n",
        "from pycocotools import mask\n",
        "from pycocotools.coco import COCO\n",
        "from pycocotools.cocoeval import COCOeval"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Mbq5qvrAp0AZ"
      },
      "source": [
        "### some functions for later"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 3,
      "metadata": {
        "id": "dRiJIhtVp0AZ"
      },
      "outputs": [],
      "source": [
        "%matplotlib inline\n",
        "def ax_decorate_box(ax):\n",
        "    [j.set_linewidth(0) for j in ax.spines.values()]\n",
        "    ax.tick_params(axis=\"both\", which=\"both\", bottom=False, top=False, \\\n",
        "               labelbottom=False, left=False, right=False, labelleft=False)\n",
        "    return ax"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 4,
      "metadata": {
        "id": "ilf4mScLAJD4"
      },
      "outputs": [],
      "source": [
        "# Plotting function\n",
        "def plot_metrics(history, metrics_list):\n",
        "    for metric in metrics_list:\n",
        "        plt.figure()\n",
        "\n",
        "        metric_values = history.history[metric]\n",
        "        val_metric_values = history.history['val_'+metric]\n",
        "\n",
        "        epochs = range(1, len(metric_values) + 1)\n",
        "        plt.plot(epochs, metric_values, 'y', label='Training '+metric)\n",
        "        plt.plot(epochs, val_metric_values, 'r', label='Validation '+metric)\n",
        "\n",
        "        plt.title('Training and Validation '+metric)\n",
        "        plt.xlabel('Epochs')\n",
        "        plt.ylabel(metric)\n",
        "        plt.legend()\n",
        "        plt.savefig(metric+'_plot.jpg')\n",
        "        plt.show()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 5,
      "metadata": {
        "id": "9QUdlxu_CSqq"
      },
      "outputs": [],
      "source": [
        "def make_coco_eval_data(image_directory,mask_directory,change_diff):\n",
        "    annotations = []\n",
        "    images = []\n",
        "    for i, image_name in enumerate(os.listdir(image_directory)):\n",
        "        image = cv2.imread(os.path.join(image_directory, image_name))\n",
        "\n",
        "        images.append({\n",
        "            \"id\": i,\n",
        "            \"file_name\": image_name,\n",
        "            \"width\": image.shape[1],\n",
        "            \"height\": image.shape[0],\n",
        "        })\n",
        "\n",
        "        mask = cv2.imread(os.path.join(mask_directory, image_name.replace('.jpg', change_diff)), cv2.IMREAD_GRAYSCALE)\n",
        "\n",
        "        for class_id in range(0, 10):  # Assuming class IDs start from 1\n",
        "            binary_mask = (mask == class_id).astype(np.uint8)\n",
        "            area = np.sum(binary_mask)\n",
        "\n",
        "            contours = measure.find_contours(binary_mask, 0.5)\n",
        "\n",
        "            segmentation = []\n",
        "            area = 0\n",
        "            for contour in contours:\n",
        "                contour = np.flip(contour, axis=1)\n",
        "                seg = contour.ravel().tolist()\n",
        "                # area += cv2.contourArea(contour)\n",
        "                if len(seg) > 4:\n",
        "                    segmentation.append(seg)\n",
        "            if len(segmentation) == 0:\n",
        "                continue\n",
        "            # area = cv2.contourArea(contour)\n",
        "\n",
        "            annotations.append({\n",
        "                \"id\": len(annotations) + 1,\n",
        "                \"image_id\": i,\n",
        "                \"category_id\": class_id,\n",
        "                \"width\": image.shape[1],\n",
        "                \"height\": image.shape[0],\n",
        "                \"score\": 0.0,\n",
        "                \"bbox\": [float(np.min(contour[:, 0])), float(np.min(contour[:, 1])),\n",
        "                        float(np.max(contour[:, 0]) - np.min(contour[:, 0])),\n",
        "                        float(np.max(contour[:, 1]) - np.min(contour[:, 1]))],\n",
        "                \"area\": area,#300,#float(maskUtils.area(maskUtils.encode(np.asfortranarray(binary_mask)))),\n",
        "                \"segmentation\": segmentation,\n",
        "                \"iscrowd\": 0,\n",
        "            })\n",
        "    return images, annotations"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "UgaDCoZgCBr0"
      },
      "source": [
        "# 01 Preprocessing"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "trPSMAIbBCaD"
      },
      "outputs": [],
      "source": [
        "drive.mount('/content/gdrive/', force_remount=True)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 22,
      "metadata": {
        "id": "ZCIfKsh3wTuf"
      },
      "outputs": [],
      "source": [
        "!unzip 10S_raw_abt.zip"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 6,
      "metadata": {
        "id": "pRhdirfJAcf6"
      },
      "outputs": [],
      "source": [
        "img_SIZE = 256\n",
        "num_CLASSES = 11"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "jfhsGvPfAsHX"
      },
      "source": [
        "### Basic Normalisation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 7,
      "metadata": {
        "id": "nufkOKKjAoxb"
      },
      "outputs": [],
      "source": [
        "def input_data_process(input_array):\n",
        "    '''converting pixel vales to [0, 1]'''\n",
        "    return input_array/255.\n",
        "\n",
        "def target_data_process(target_array):\n",
        "    return keras.utils.to_categorical(target_array, num_classes=num_CLASSES)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "_JUkuavUXJes"
      },
      "source": [
        "Uncomment this one if you want to use it with the 5S dataset."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 14,
      "metadata": {
        "id": "Ao_6u2gWBVka"
      },
      "outputs": [],
      "source": [
        "# ## this one is for our 5 Shape dataset where some of the particles class lables\n",
        "# ## need to be remapped\n",
        "\n",
        "\n",
        "# def input_data_process(input_array):\n",
        "#     '''converting pixel vales to [0, 1]'''\n",
        "#     return input_array/255\n",
        "\n",
        "# def target_data_process(target_array):\n",
        "#     target_array[target_array == 8] = 4\n",
        "#     target_array[target_array == 9] = 5\n",
        "#     return keras.utils.to_categorical(target_array, num_classes=num_CLASSES)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "GQJFN1UOAzw4"
      },
      "source": [
        "### Keras VGG16 Normalisation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "uh1PwhSFBFLn"
      },
      "outputs": [],
      "source": [
        "# def input_data_process(input_array):\n",
        "#     return tf.keras.applications.vgg16.preprocess_input(input_array, data_format=None)\n",
        "\n",
        "# def target_data_process(target_array):\n",
        "#     return keras.utils.to_categorical(target_array, num_classes=num_CLASSES)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "3fEKAuciAXCn"
      },
      "outputs": [],
      "source": [
        "# def input_data_process_vis(input_array):\n",
        "#     '''converting pixel vales to [0, 1]'''\n",
        "#     return input_array/255\n",
        "\n",
        "# def target_data_process_vis(target_array):\n",
        "#     return keras.utils.to_categorical(target_array, num_classes=num_CLASSES)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "I1NWkRpn-uRN"
      },
      "source": [
        "\n",
        "\n",
        "### load images"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "W848n6qwX2ML"
      },
      "outputs": [],
      "source": [
        "dataset_mode = 'abt' # change abt to raw if you want to train on only the raw data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "TwBlFOawIu1k"
      },
      "outputs": [],
      "source": [
        "# paths to your dataset\n",
        "path_train_img_raw = '/content/img_train/'\n",
        "path_train_mask_raw = '/content/msk_train/'\n",
        "path_train_img_a = '/content/iabt_img_train/'\n",
        "path_train_mask_a = '/content/iabt_msk_train/'\n",
        "\n",
        "path_valid_img = '/content/img_val/'\n",
        "path_valid_mask = '/content/msk_val/'\n",
        "\n",
        "path_test_img = '/content/img_test/'\n",
        "path_test_mask = '/content/msk_test/'\n",
        "\n",
        "train_input_names_raw = np.array(sorted(glob(path_train_img_raw +'*.jpg')))\n",
        "train_label_names_raw = np.array(sorted(glob(path_train_mask_raw +'*.png')))\n",
        "\n",
        "train_input_names_a = np.array(sorted(glob(path_train_img_a +'*.jpg')))\n",
        "train_label_names_a = np.array(sorted(glob(path_train_mask_a +'*.png')))\n",
        "\n",
        "if dataset_mode == 'raw':\n",
        "  train_input_names_a = np.array(sorted(glob(path_train_img_raw +'*.jpg')))\n",
        "  train_label_names_a = np.array(sorted(glob(path_train_mask_raw +'*.png')))\n",
        "\n",
        "train_input_names = np.concatenate((train_input_names_raw,\n",
        "                                    train_input_names_a),\n",
        "                                   axis=0)\n",
        "train_label_names = np.concatenate((train_label_names_raw,\n",
        "                                    train_label_names_a),\n",
        "                                   axis=0)\n",
        "\n",
        "valid_input_names = np.array(sorted(glob(path_valid_img +'*.jpg')))\n",
        "valid_label_names = np.array(sorted(glob(path_valid_mask +'*.png')))\n",
        "test_input_names = np.array(sorted(glob(path_test_img+'*.jpg')))\n",
        "test_label_names = np.array(sorted(glob(path_test_mask+'*.png')))\n",
        "L_train = len(train_input_names)\n",
        "\n",
        "print(\"Training:validation = {}:{}:{}\".format(len(train_input_names),\n",
        "                                              len(valid_input_names),\n",
        "                                              len(test_label_names)))"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 20,
      "metadata": {
        "id": "3WM91JHNiS7E"
      },
      "outputs": [],
      "source": [
        "a_shuffled, b_shuffled = shuffle(train_input_names, train_label_names)\n",
        "### SHUFFLE\n",
        "train_input = input_data_process(utils.image_to_array(a_shuffled,\n",
        "                                                      size=img_SIZE,\n",
        "                                                      channel=3))\n",
        "train_label = target_data_process(utils.image_to_array(b_shuffled,\n",
        "                                                       size=img_SIZE,\n",
        "                                                       channel=1))"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Uq3pS926p0Ab"
      },
      "outputs": [],
      "source": [
        "valid_input = input_data_process(utils.image_to_array(valid_input_names,\n",
        "                                                      size=img_SIZE,\n",
        "                                                      channel=3))\n",
        "valid_label = target_data_process(utils.image_to_array(valid_label_names,\n",
        "                                                       size=img_SIZE,\n",
        "                                                       channel=1))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Q97uRXoF-0wZ"
      },
      "source": [
        "### visualise some of the training data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "eXlLPDdMRCp8"
      },
      "outputs": [],
      "source": [
        "\n",
        "\n",
        "def plot_random_images(input_images, label_images, num_images):\n",
        "    indices = np.random.choice(len(input_images), num_images)\n",
        "    for idx in indices:\n",
        "        label_class = np.argmax(train_label[idx], axis=-1)\n",
        "        plt.figure(figsize=(10,5))\n",
        "        plt.subplot(1, 2, 1)\n",
        "        plt.imshow(train_input[idx])\n",
        "        plt.title('Input Image')\n",
        "        plt.subplot(1, 2, 2)\n",
        "        plt.imshow(label_class, cmap='gray')\n",
        "        plt.title('Label')\n",
        "        plt.show()\n",
        "        unique_classes = np.unique(label_class)\n",
        "        print(f\"Unique classes represented in the label image: {unique_classes}\")\n",
        "plot_random_images(train_input_names, train_label_names, num_images=10)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "k06nxxYOwxg4"
      },
      "source": [
        "### calc weights for imbalance\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "RjuxCIx1Ey2r"
      },
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "\n",
        "num_classes = train_label.shape[-1]\n",
        "class_counts = np.sum(train_label, axis=(0,1,2))\n",
        "print('Pixel per class:')\n",
        "print(class_counts)\n",
        "\n",
        "# Calculate class weights\n",
        "class_weights = 1. / class_counts  # Inverse of the number of pixels\n",
        "class_weights = class_weights / np.max(class_weights)\n",
        "print('Class weights for Dice Loss')\n",
        "print(class_weights)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "9UUpk70e2qIX"
      },
      "source": [
        "# 02 Model selection"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "cCT0KDY__-rW"
      },
      "outputs": [],
      "source": [
        "## unet\n",
        "model = models.unet_2d((img_SIZE, img_SIZE, 3), filter_num=[64, 128, 256, 512, 1024], n_labels=num_CLASSES, stack_num_down=2, stack_num_up=2,\n",
        "            activation='ReLU', output_activation='Softmax', batch_norm=False, pool=True, unpool=True,\n",
        "            backbone='VGG16', weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='unet')"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "hA9eRRqZ2uMq"
      },
      "outputs": [],
      "source": [
        "## attent UNet\n",
        "model = models.att_unet_2d((img_SIZE, img_SIZE, 3), filter_num=[64, 128, 256, 512, 1024], n_labels=num_CLASSES,\n",
        "                           stack_num_down=2, stack_num_up=2, activation='ReLU',\n",
        "                           atten_activation='ReLU', attention='add', output_activation='Softmax',\n",
        "                           batch_norm=False, pool=True, unpool=True,\n",
        "                           backbone='VGG16', weights='imagenet',\n",
        "                           freeze_backbone=True, freeze_batch_norm=True,\n",
        "                           name='attunet')"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "REMZnPkJ20Ym"
      },
      "outputs": [],
      "source": [
        "## UNet+++\n",
        "model = models.unet_3plus_2d((img_SIZE, img_SIZE, 3), n_labels=num_CLASSES, filter_num_down=[64, 128, 256, 512, 1024], filter_num_skip='auto', filter_num_aggregate='auto',\n",
        "                  stack_num_down=2, stack_num_up=2, activation='ReLU', output_activation='Softmax',\n",
        "                  batch_norm=False, pool=True, unpool=True, deep_supervision=False,\n",
        "                  backbone='VGG16', weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='unet3plus')\n",
        "\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "VtMD5w8mYVm2"
      },
      "source": [
        "### compile the model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "WnRtmvuw9c23"
      },
      "outputs": [],
      "source": [
        "dice_loss = sm.losses.DiceLoss(class_weights=class_weights)\n",
        "total_loss = dice_loss"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "wRYTfD6o9vvs"
      },
      "outputs": [],
      "source": [
        "metrics = [\n",
        "    sm.metrics.IOUScore(threshold=0.5)\n",
        "    ]"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "XgnRs7UJATFz"
      },
      "outputs": [],
      "source": [
        "model.compile(loss=total_loss,\n",
        "              optimizer=keras.optimizers.Adam(learning_rate=0.00025),\n",
        "              metrics=metrics)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "dPhoFOF9b2p6"
      },
      "source": [
        "# 03 Train\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "eePnjtyGbzPA"
      },
      "outputs": [],
      "source": [
        "N_epoch = 64\n",
        "N_batch = 8\n",
        "\n",
        "model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(\n",
        "    filepath='/content/tmp1/',\n",
        "    save_weights_only=True,\n",
        "    monitor='val_iou_score',\n",
        "    mode='max',\n",
        "    save_best_only=True)\n",
        "\n",
        "# Train your model\n",
        "history1 = model.fit(\n",
        "    x=train_input,\n",
        "    y=train_label,\n",
        "    batch_size=N_batch,\n",
        "    epochs=N_epoch,\n",
        "    shuffle=True,\n",
        "    validation_data=(valid_input, valid_label),\n",
        "    callbacks=[model_checkpoint_callback])\n",
        "\n",
        "# load the best performing model\n",
        "model.load_weights('/content/tmp1/')\n",
        "y_pred = model.predict(valid_input)\n",
        "val_iou = np.mean(losses.iou_seg(valid_label, y_pred))\n",
        "iou_per_class = []\n",
        "for i in range(num_CLASSES):\n",
        "    iou = np.mean(losses.iou_seg(valid_label[..., i], y_pred[..., i]))\n",
        "    iou_per_class.append(iou)\n",
        "print('IoU loss per class:', iou_per_class)\n",
        "\n",
        "\n",
        "########## only raw\n",
        "train_input = input_data_process(utils.image_to_array(\n",
        "                                                    train_input_names_raw,\n",
        "                                                    size=img_SIZE, channel=3))\n",
        "train_label = target_data_process(utils.image_to_array(\n",
        "                                                    train_label_names_raw,\n",
        "                                                    size=img_SIZE, channel=1))\n",
        "\n",
        "model.load_weights('/content/tmp1/')\n",
        "K.set_value(model.optimizer.learning_rate, 0.000025)\n",
        "N_epoch = 32\n",
        "N_batch = 8\n",
        "\n",
        "model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(\n",
        "    filepath='/content/tmp/',\n",
        "    save_weights_only=True,\n",
        "    monitor='val_iou_score',\n",
        "    mode='max',\n",
        "    save_best_only=True)\n",
        "\n",
        "# Train your model\n",
        "history2 = model.fit(\n",
        "    x=train_input,\n",
        "    y=train_label,\n",
        "    batch_size=N_batch,\n",
        "    epochs=N_epoch,\n",
        "    shuffle=True,\n",
        "    validation_data=(valid_input, valid_label),\n",
        "    callbacks=[model_checkpoint_callback])\n",
        "\n",
        "# load the best performing model\n",
        "model.load_weights('/content/tmp/')\n",
        "y_pred = model.predict(valid_input)\n",
        "val_iou = np.mean(losses.iou_seg(valid_label, y_pred))\n",
        "iou_per_class = []\n",
        "for i in range(num_CLASSES):\n",
        "    iou = np.mean(losses.iou_seg(valid_label[..., i], y_pred[..., i]))\n",
        "    iou_per_class.append(iou)\n",
        "print('IoU loss per class:', iou_per_class)\n",
        "\n",
        "\n",
        "\n",
        "#########################################\n",
        "model.load_weights('/content/tmp/')\n",
        "K.set_value(model.optimizer.learning_rate, 0.0000025)\n",
        "N_epoch = 16\n",
        "N_batch = 8\n",
        "\n",
        "model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(\n",
        "    filepath='/content/tmp/',\n",
        "    save_weights_only=True,\n",
        "    monitor='val_iou_score',\n",
        "    mode='max',\n",
        "    save_best_only=True)\n",
        "\n",
        "# Train your model\n",
        "history3 = model.fit(\n",
        "    x=train_input,\n",
        "    y=train_label,\n",
        "    batch_size=N_batch,\n",
        "    epochs=N_epoch,\n",
        "    shuffle=True,\n",
        "    validation_data=(valid_input, valid_label),\n",
        "    callbacks=[model_checkpoint_callback])\n",
        "\n",
        "# load the best performing model\n",
        "model.load_weights('/content/tmp/')\n",
        "y_pred = model.predict(valid_input)\n",
        "val_iou = np.mean(losses.iou_seg(valid_label, y_pred))\n",
        "iou_per_class = []\n",
        "for i in range(num_CLASSES):\n",
        "    iou = np.mean(losses.iou_seg(valid_label[..., i], y_pred[..., i]))\n",
        "    iou_per_class.append(iou)\n",
        "print('IoU loss per class:', iou_per_class)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "HABcbfojnMkW"
      },
      "outputs": [],
      "source": [
        "plot_metrics(history1, metrics_list=['loss', 'iou_score'])"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "yy5vB7TaPFgi"
      },
      "outputs": [],
      "source": [
        "plot_metrics(history2, metrics_list=['loss', 'iou_score'])"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "px7-LfQhPFRU"
      },
      "outputs": [],
      "source": [
        "plot_metrics(history3, metrics_list=['loss', 'iou_score'])"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "h6iGWIKxc7KT"
      },
      "source": [
        "# 04 Evaluate"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Wby401vVon9B"
      },
      "outputs": [],
      "source": [
        "test_metrics = model.evaluate(x=test_input, y=test_label, batch_size=N_batch)\n",
        "\n",
        "print(\"\\nEvaluation results:\")\n",
        "for name, value in zip(model.metrics_names, test_metrics):\n",
        "    print(f\"{name}: {value}\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "MOLBMPoWgyZg"
      },
      "outputs": [],
      "source": [
        "\n",
        "y_pred = model.predict(test_input)\n",
        "val_iou = np.mean(losses.iou_seg(test_label, y_pred))\n",
        "iou_per_class = []\n",
        "\n",
        "for i in range(num_CLASSES):\n",
        "    iou = np.mean(losses.iou_seg(test_label[..., i], y_pred[..., i]))\n",
        "    iou_per_class.append(iou)\n",
        "print('IoU loss per class test:', iou_per_class)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "bgAUbg_1DJFA"
      },
      "outputs": [],
      "source": [
        "\n",
        "# Predict the segmentation for this sample\n",
        "prediction = y_pred\n",
        "# test_input_vis = input_data_process_vis(utils.image_to_array(test_input_names, size=img_SIZE, channel=3))\n",
        "for i in range(len(test_input)):\n",
        "  # Convert the prediction to a single-channel segmentation mask\n",
        "  pred_mask = np.argmax(prediction[i], axis=-1)\n",
        "  # Convert the ground truth to a single-channel segmentation mask\n",
        "  gt_mask = np.argmax(test_label[i], axis=-1)\n",
        "  plt.imsave('results/'+str(i) +'_.jpg',test_input[i])\n",
        "  cv2.imwrite('results/'+str(i) +'_dt.png', pred_mask)\n",
        "  cv2.imwrite('results/'+str(i) +'_gt.png', gt_mask)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "p1y_0OcnZUZU"
      },
      "source": [
        "Use this one if you are using VGG normalisation."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "lFSmQbTwpNyy"
      },
      "outputs": [],
      "source": [
        "### use this one if you are using VGG normalization\n",
        "\n",
        "\n",
        "# # Predict the segmentation for this sample\n",
        "# prediction = model.predict(test_input)\n",
        "# test_input_vis = input_data_process_vis(utils.image_to_array(test_input_names, size=img_SIZE, channel=3))\n",
        "# for i in range(len(test_input_vis)):\n",
        "#   # Convert the prediction to a single-channel segmentation mask\n",
        "#   pred_mask = np.argmax(prediction[i], axis=-1)\n",
        "#   # Convert the ground truth to a single-channel segmentation mask\n",
        "#   gt_mask = np.argmax(test_label[i], axis=-1)\n",
        "#   plt.imsave('results/'+str(i) +'_.jpg',test_input_vis[i])\n",
        "#   cv2.imwrite('results/'+str(i) +'_dt.png', pred_mask) #pred_mask\n",
        "#   cv2.imwrite('results/'+str(i) +'_gt.png', gt_mask) #pred_mask"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "6q39YcklZYaS"
      },
      "source": [
        "### preparing for MS-COCO evaluatio\n",
        "since the coco library needs a specific format in order to evaluate\n",
        "this is a \"poor mans implementation\" for doing so."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "fF3HUV6Ie6Wd"
      },
      "outputs": [],
      "source": [
        "\n",
        "image_directory_GT = 'results'\n",
        "mask_directory_GT = 'results'\n",
        "output_file_GT = 'GT_anno.json'\n",
        "change_diff_GT = 'gt.png'\n",
        "\n",
        "\n",
        "image_directory_DT = 'results'\n",
        "mask_directory_DT = 'results'\n",
        "output_file_DT = 'results_GT_anno.json'\n",
        "change_diff_DT = 'dt.png'\n",
        "\n",
        "# Save everything to the annotation file\n",
        "images, annotations = make_coco_eval_data(image_directory_GT,mask_directory_GT,change_diff_GT)\n",
        "with open(output_file_GT, 'w') as f:\n",
        "    json.dump({\n",
        "        \"images\": images,\n",
        "        \"annotations\": annotations,\n",
        "        \"categories\": [{\"id\": i, \"name\": str(i)} for i in range(0, 10)],\n",
        "    }, f)\n",
        "images, annotations2 = make_coco_eval_data(image_directory_DT,mask_directory_DT,change_diff_DT)\n",
        "with open(output_file_DT, 'w') as f:\n",
        "    json.dump({\"annotations\":annotations2\n",
        "    }, f)\n",
        "with open('/content/results_GT_anno.json', 'r') as f:\n",
        "    data = f.read()\n",
        "    tmp = data[16:-1]\n",
        "\n",
        "with open('new_results_GT_anno.json', 'w') as f:\n",
        "    f.write(tmp)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "eanitDcadOVi"
      },
      "outputs": [],
      "source": [
        "cocoGt=COCO('/content/GT_anno.json')\n",
        "cocoDt=cocoGt.loadRes('/content/new_results_GT_anno.json')"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "qTj41Z3XdOOf"
      },
      "outputs": [],
      "source": [
        "cocoEval = COCOeval(cocoGt,cocoDt,'segm')\n",
        "# cocoEval.params.imgIds  = 1\n",
        "cocoEval.evaluate()\n",
        "cocoEval.accumulate()\n",
        "cocoEval.summarize()"
      ]
    }
  ],
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "collapsed_sections": [
        "VDjeb6uuZp_u",
        "Mbq5qvrAp0AZ",
        "UgaDCoZgCBr0",
        "jfhsGvPfAsHX",
        "GQJFN1UOAzw4",
        "eFcr5_WPIwGQ",
        "I1NWkRpn-uRN",
        "Q97uRXoF-0wZ",
        "k06nxxYOwxg4",
        "9UUpk70e2qIX",
        "VtMD5w8mYVm2",
        "dPhoFOF9b2p6",
        "h6iGWIKxc7KT"
      ],
      "provenance": []
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.6.8"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}